In [24]:
import cv2
import numpy as np
import os
import sys
sys.path.append(os.path.abspath('../'))
import pandas as pd
import datetime
import random
from utilities.helper_functions import *
import config
import matplotlib.pyplot as plt

In [25]:
## Imports array of videos and names of videos from a folder
# @param path: path to folder containing videos
# Returns a list of videos and a list of names
def load_videos(path, regex = None):
    videos = []
    names = []
    for file in os.listdir(path):
        if file.endswith('.mp4'):
            if regex is not None:
                if not re.search(regex, file):
                    continue
            names.append(path.split('/')[-1] + file.split('.')[0])
            video_capture = cv2.VideoCapture(path + '/' + file)
            video = []
            while True:
                ret, frame = video_capture.read()
                if not ret:
                    break
                # convert to grayscale
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                video.append(frame)
            videos.append(np.array(video))
    return videos, names


In [26]:
video_paths = [config.video_output_path[:-1] + '_all/' + trial for trial in os.listdir(config.video_output_path[:-1] + '_all/') ]


In [27]:
video_paths

['/mnt/teams/Tsuchitori/social_interaction_trials_all/20230818120023',
 '/mnt/teams/Tsuchitori/social_interaction_trials_all/20230814161151',
 '/mnt/teams/Tsuchitori/social_interaction_trials_all/20230822152926',
 '/mnt/teams/Tsuchitori/social_interaction_trials_all/20230812150714',
 '/mnt/teams/Tsuchitori/social_interaction_trials_all/20230814153947',
 '/mnt/teams/Tsuchitori/social_interaction_trials_all/20230810171031',
 '/mnt/teams/Tsuchitori/social_interaction_trials_all/20230712151237',
 '/mnt/teams/Tsuchitori/social_interaction_trials_all/20230816103729',
 '/mnt/teams/Tsuchitori/social_interaction_trials_all/20230823103645',
 '/mnt/teams/Tsuchitori/social_interaction_trials_all/20230812144422',
 '/mnt/teams/Tsuchitori/social_interaction_trials_all/20230818121524',
 '/mnt/teams/Tsuchitori/social_interaction_trials_all/20230718154037',
 '/mnt/teams/Tsuchitori/social_interaction_trials_all/20230810161151',
 '/mnt/teams/Tsuchitori/social_interaction_trials_all/20230831104835',
 '/mnt

In [28]:
## Take each a video and then subtract each frame from the next frame to get the next movement
def calculate_movement(video):
    movement = []
    for i in range(len(video) - 1):
        movement.append(video[i + 1].astype('int16') - video[i].astype('int16'))
    return np.abs((np.array(movement) + 256) // 2).astype('uint8')


In [29]:
def extract_movement_frames(video):
    movement = []
    for i in range(len(video) - 1):
        movement.append(np.abs(np.sum(video[i + 1].astype('int16') - video[i].astype('int16'))))
    # take a moving average of the movement
    try:
        movement = np.convolve(movement, np.ones(100) / 100, mode='same')
    except (ValueError):
        movement = np.zeros(len(movement))
    return movement


In [30]:
# iterate through all the videos, extract interactions, and then save the video of the interactions to a folder
for path in video_paths:
    videos, names = load_videos(path, regex = '^((?!_full_vid).)*$')
    for video, name in zip(videos, names):
        movement = extract_movement_frames(video)
        if len(movement) < 2:
            continue
        if np.max(movement) > 1000:
            # find the first frame where the movement is greater than 1000
            start = np.where(movement > 1000)[0][0]
            # find the last frame where the movement is greater than 1000
            end = np.where(movement > 1000)[0][-1]
            movemnent_extracted_video = video[start:end]
            # save the video to the movement_extracted folder


            helper_functions.save_video(movemnent_extracted_video, config.movement_extracted_output_path[:-1] + '_all/' + name.split('_')[0] + '/' + name + '_start-' + str(start) + '_end-' + str(end) + '.mp4')
            helper_functions.save_video(video, config.movement_extracted_output_path[:-1] + '_all/' + name.split('_')[0] + '/' + name + '_full_length' + '.mp4')

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x561c3c45e200] moov atom not found


In [31]:
video_paths = [config.video_output_path[:-1] + '/' + trial for trial in os.listdir(config.video_output_path[:-1] + '/') ]
video_paths


['/mnt/teams/Tsuchitori/social_interaction_trials/20230818120023',
 '/mnt/teams/Tsuchitori/social_interaction_trials/20230814161151',
 '/mnt/teams/Tsuchitori/social_interaction_trials/20230714161545',
 '/mnt/teams/Tsuchitori/social_interaction_trials/20230822152926',
 '/mnt/teams/Tsuchitori/social_interaction_trials/20230718144045',
 '/mnt/teams/Tsuchitori/social_interaction_trials/20230812150714',
 '/mnt/teams/Tsuchitori/social_interaction_trials/20230713094734',
 '/mnt/teams/Tsuchitori/social_interaction_trials/20230718161229',
 '/mnt/teams/Tsuchitori/social_interaction_trials/20230814153947',
 '/mnt/teams/Tsuchitori/social_interaction_trials/20230714114827',
 '/mnt/teams/Tsuchitori/social_interaction_trials/20230810171031',
 '/mnt/teams/Tsuchitori/social_interaction_trials/20230712151237',
 '/mnt/teams/Tsuchitori/social_interaction_trials/20230816103729',
 '/mnt/teams/Tsuchitori/social_interaction_trials/20230823103645',
 '/mnt/teams/Tsuchitori/social_interaction_trials/202307141416

In [32]:
# iterate through all the videos, extract interactions, and then save the video of the interactions to a folder
for path in video_paths:
    videos, names = load_videos(path, regex = '^((?!_full_vid).)*$')
    for video, name in zip(videos, names):
        movement = extract_movement_frames(video)
        if len(movement) < 2:
            continue
        if np.max(movement) > 1000:
            # find the first frame where the movement is greater than 1000
            start = np.where(movement > 1000)[0][0]
            # find the last frame where the movement is greater than 1000
            end = np.where(movement > 1000)[0][-1]
            movemnent_extracted_video = video[start:end]
            # save the video to the movement_extracted folder

            helper_functions.save_video(movemnent_extracted_video, config.movement_extracted_output_path[:-1] + '/' + name.split('_')[0] + '/' + name + '_start-' + str(start) + '_end-' + str(end) + '.mp4')
            helper_functions.save_video(video, config.movement_extracted_output_path[:-1] + '/' + name.split('_')[0] + '/' + name + '_full_length' + '.mp4')